In [ ]:
import pymongo
from pymongo import MongoClient
import os

In [ ]:
db_name: str = "myntra"
collection_name: str = "myntra-items-offline"

In [ ]:
client = MongoClient(os.environ["DB_CONNECTION_STRING"])
collection = client[db_name][collection_name]

In [ ]:
text_search = 'dress'
text_search_index_name = 'lexical_text_search_index'
min_price=0
max_price=10_000
min_rating=0
n=10

In [ ]:
def lexical_search(text_search: str, text_search_index_name: str) -> list[dict]:
    return [
        {
            "$search": {
                "index": text_search_index_name,
                "text": {
                    "query": text_search,
                    "path": "name",
                },
            }
        }
    ]


In [ ]:
def match_on_metadata(
    min_price: int,
    max_price: int,
    min_rating: float,
    n: int,
    categories: list[str] | None = None,
    colors: list[str] | None = None,
    seasons: list[str] | None = None,
) -> list[dict]:
    match_spec = {
        "price": {
            "$gte": min_price,
            "$lte": max_price,
        },
        "rating": {"$gte": min_rating},
    }
    if categories:
        match_spec["category"] = {"$in": categories}
    if colors:
        match_spec["color"] = {"$in": colors}
    if seasons:
        match_spec["season"] = {"$in": seasons}

    return [
        {
            "$match": match_spec,
        },
        {"$limit": n},
    ]


In [ ]:
pipeline = []
if text_search.strip():
    pipeline.extend(
        lexical_search(
            text_search=text_search,
            text_search_index_name=text_search_index_name,
        )
    )

pipeline.extend(
    match_on_metadata(
        min_price=min_price,
        max_price=max_price,
        min_rating=min_rating,
        n=n,
    )
)

records = collection.aggregate(pipeline)
results = [
    (record["img"].split(";")[-1].strip(), record["name"])
    # async 
    for record in records
]

n_results = len(results)

In [ ]:
results

In [ ]:
n_results